In [1]:
%load_ext autoreload

In [48]:
%autoreload
import sys
sys.path.insert(0, '../')

In [49]:
from src.utilities.mluar_utils import *

In [43]:
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset, Dataset, load_from_disk
import numpy as np
from einops import rearrange, reduce, repeat
import torch
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import pyplot as plt
import math
import pandas as pd
import pickle as pkl

pd.set_option('display.max_colwidth', None)


In [5]:
MULTI_LUAR_PATH =  "/mnt/swordfish-pool2/milad/multi-luar-reddit-model/"
LUAR_PATH =  "/mnt/swordfish-pool2/nikhil/LUAR/pretrained_weights/LUAR-MUD/"

In [6]:
# Load models
multiluar_model = AutoModel.from_pretrained(MULTI_LUAR_PATH, trust_remote_code=True)
luar_model = AutoModel.from_pretrained(LUAR_PATH, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("rrivera1849/LUAR-MUD")

In [52]:
# Load data
data_path = '/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_perGenre-HRS2.1/TA2/hrs_06-27-24_english_perGenre-HRS2.1/data/hrs_06-27-24_english_perGenre-HRS2.1_TA2_input'
ground_truth_path = '/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_perGenre-HRS2.1/TA2/hrs_06-27-24_english_perGenre-HRS2.1/groundtruth/hrs_06-27-24_english_perGenre-HRS2.1_TA2'
hiatus_data, _, _ = load_aa_data(data_path, ground_truth_path)

Loading:  /mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_perGenre-HRS2.1/TA2/hrs_06-27-24_english_perGenre-HRS2.1/data/hrs_06-27-24_english_perGenre-HRS2.1_TA2_input


In [53]:
# keep authors with only more than one text
authors_with_multiple_texts = [x[0] for x in hiatus_data.authorID.value_counts().to_dict().items() if x[1] > 1]
hiatus_data = hiatus_data[hiatus_data.authorID.isin(authors_with_multiple_texts)]

In [54]:
hiatus_data = hiatus_data.sample(2000)

In [55]:
hiatus_data_texts = hiatus_data.fullText.tolist()
labels = hiatus_data['authorID'].tolist()

# Compute embeddings
max_seq_length = 736
hiatus_data_embeddings = get_luar_embeddings(hiatus_data_texts, multiluar_model, tokenizer, max_length=max_seq_length, batch_size=1, is_multi_luar=True)

In [10]:
#np.max([len(x.split()) for x in hiatus_data.fullText.tolist()])

### Experiment Design

- For each layer, we find pairs of ground-truth texts written by the same author where the corresponding layer's embedding scored them high compared to other layers
- For each layer, we take a sample of these pairs of texts and prompt ChatGPT to find which linguistic level they are similar

#### Step 1:

In [56]:
# Compute Multi-luar similarity matrices at every layer and average of the layers
muti_luar_layers_sims = [compute_similarities(hiatus_data_embeddings, hiatus_data_embeddings, layer=i) for i in range(7)]
#muti_luar_layers_sims.append(compute_similarities(hiatus_data_embeddings, hiatus_data_embeddings, layer=None))
muti_luar_layers_sims = np.stack(muti_luar_layers_sims)

In [57]:
# Compute significant pairs to layers
layer_to_sig_pairs = {layer: extract_sig_pairs_for_layer(hiatus_data_texts, muti_luar_layers_sims, layer) for layer in range(7)}

In [59]:
layer_x_pairs = []
for layer in [0, 1, 2, 3, 4, 5, 6]:
    print(layer, len(layer_to_sig_pairs[layer]))
    sample_of_pairs = layer_to_sig_pairs[layer][:10]
    layer_x_pairs += [{'text-1': x[0], 'text-2': x[1], 'z-score': x[2], 'layer-sim': x[3], 'layer': layer} for x in sample_of_pairs]
layer_x_pairs_df = pd.DataFrame(layer_x_pairs)

0 9984
1 3
2 20
3 56
4 1
5 5
6 299


In [60]:
layer_x_pairs_df[layer_x_pairs_df.layer == 1].head()

,text-1,text-2,z-score,layer-sim,layer
10,"Knots\n\nWatched as part of The Tara Reid Mission\n\nThe Film\nThere's a moment in Knots where <PERSON> character tells his friend that his prolific cheating on his girlfriend is okay because a) Her work takes her away a lot, and b) Sex with other women further solidifies his feeling that his girlfriend is 'the one' and that the sex with her after he's cheated is brilliant. It's this kind of mean spiritedness that defines the film, from the characters to the situations they create for themselves.\n\nSupposedly a comedy from the male perspective, very few of the characters ring true. The men themselves are hapless, bumbling idiots, so sure of their masculinity that they are unable to accept their (many) flaws, whilst the women are either screaming harpies, dumb conquests or manipulative ice maidens.\n\nIt's not all bad though. There are a few genuinely amusing moments, and <PERSON>, who is the one genuinely nice guy, does well with what little the script gives him.\n\nIf this is what modern relationships are like, count me out.\n\nHow's <PERSON>?\nNot appearing until around the 42 minute mark, <PERSON>'s <PERSON> was the only other character I didn't destest. The scenes with her and <PERSON>'s <PERSON> are sweet, and it was nice seeing their relationship blossom, although more time is spent with the other, more hateful characters. <PERSON>'s well documented of screen partying seems to have taken it's toll though, as she sounds like she's smoked a pack of cigarettes every time she opens her mouth.",DNRs\n\nSeriously what is the fucking point of DNRing someone if the other people keep letting them come back. As well as the owner and gm???????\n\nFor instance one guy stayed a few months back had gotten into it with the owner and has an aggressive pitbull (no not all pitties are aggressive but this one is)\n\nThe owner told me to add him to the long list of dnr. Because they smoked in the room has the aggressive dog and trashed the room.\n\nThe owner told him not to come back literally to his face.\n\nSo i come in after being off yesterday and lowandbehold he has come back. 🙃\n\nI say something to the owner but they act like they dont give two shits\n\nSo what was the whole big deal last time then?\n\nHe comes to the desk today wanting to basically to the standard room and upgrade cause hes a starrbellysneech memeber.\n\n(You dont get an upgrade on booked out nights and its only to the suite not studio if its avaliable)\n\nAnd of couse they gotta do this at 11 right at check out time. With a fuck ton of shit packed into their room on a (1 night stay)\n\nHousekeeping knocks like 4 times they keeps shewing them away so the owner tells the guy hey its 1130 yall need move to the new room.\n\nOf cousrse he starts to argue saying i didnt let him check back in Which is a lie.\n\nHis phone signal was shit so he had to go outside and book it cause he didnt want to pay the higher rate.\n\nFinally it came through and i check him in within like 2 min. So no dude it wasnt me. You knew what time it was your reservation due to your shitty ass broken phone was the one who was fucking it up not me.\n\nHe had the dog and it still caost him like 180 woth the pet fee. And i charged it out before he left my desk. He is one of those who will turn their card off. Im making sure we got the money.\n\nAnd i blocked out the last standard room on pourpose.\n\nAnyway i jsut had a conversation that with the owner as to why... he said cause its been slow and he needs the money. So he got rid of the list that he himself told me to make.\n\nWe have not been slow this week almost every night we have been booked out. So spare me the bull shit Nile.\n\nI just threw my hands up and said hey om just letting you know. Is what it is.\n\nYou wanna argue with the same dumb asses every time go ahead.\n\n( i am quiet quitting anyways)\n\nI cant take the back and fourth double standard anymore. I do my job and its never good enough.\

In [28]:
layer_x_pairs_df.to_json('../data/layer_to_pairs_signficance.json')

#### Step 2:

In [8]:
from datadreamer import DataDreamer
from datadreamer.llms import HFTransformers, ParallelLLM, OpenAI
from datadreamer.steps import DataFromPrompt, ProcessWithPrompt,  HFHubDataSource, DataSource, zipped, concat
from functools import partial
from transformers import QuantoConfig
from datasets import concatenate_datasets, load_dataset
import json


model = OpenAI(model_name="gpt-4", api_key='sk-proj-zTbZNk16Ik1pZnqLn38ZT3BlbkFJImq3pd7widkr7RzsC771') #kathy's lab
# model = HFTransformers(
#                 "meta-llama/Meta-Llama-3-8B-Instruct",
#                 quantization_config=QuantoConfig(weights="int8"),
#                 device=0,
#                 device_map="cuda",
#             )
# model.config.pad_token_id = model.config.eos_token_id

def gen_from_iterable_dataset(iterable_ds):
    yield from iterable_ds

def evaluate_text_similarities(data_path, document_pairs, linguistic_lvl_name, linguistic_lvl_desc):
    instruction = "Given the two Documents below, rate their <linguistic_lvl> on a scale from 1 to 5: Score 1 equals very low similarity and score 5 equals high similarity. First, give reasons for your score and then output the score. The output should be in the following format: {\"reasons\": \"explain your rating\",  \"score\": \"<json integer>\"}"
    instruction = instruction.replace("<linguistic_lvl>", linguistic_lvl_desc)
    
    with DataDreamer(data_path):
        datasource = DataSource('documents', Dataset.from_list(document_pairs))
        datasource = datasource.map(lambda row: {'inputs': 'Document 1:\n {} \n Document 2:\n {}'.format(row['text-1'], row['text-2'])}, auto_progress=False)
        ds_focus_questions = ProcessWithPrompt(
          "{} describe text similarity".format(linguistic_lvl_name),
          inputs={"inputs": datasource.output["inputs"]},
          args={
             "llm": model,
             "n": 1,
             "instruction": instruction
          },
          outputs={"generations": linguistic_lvl_name},
        ).select_columns([linguistic_lvl_name])
        
        zipped_step = zipped(datasource, ds_focus_questions)

        results_iter = zipped_step.output.dataset
        results_ds   = Dataset.from_generator(partial(gen_from_iterable_dataset, results_iter))

        return results_ds

In [10]:
layer_x_pairs_df = pd.read_json('../data/layer_to_pairs_signficance.json')

In [11]:
layer_x_pairs_df.layer.value_counts()

0    10
2    10
3    10
6    10
4     4
1     3
5     3
Name: layer, dtype: int64

In [12]:
# Given the two Documents below, rate their <linguistic_lvl> on a scale from 1 to 5: Score 1 equals very low similarity and score 5 equals high similarity. First, give reasons for your score and then output the score in the following output format: {\"reasons\": \"explain your rating\",  \"score\": \"<json integer>\"}
ling_phenomena = {
    'syntax': 'syntactic similarity',
    'semantic': 'semantic similarity',
    'lexical': 'lexical similarity',
    'discourse': 'discourse similarity'
}

In [20]:
layer_x_pairs = [row.to_dict() for idx, row in layer_x_pairs_df.iterrows()]

In [23]:
results = []
for key, val in ling_phenomena.items():
    results.append(evaluate_text_similarities('./output', layer_x_pairs, key, val))

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./output
[ 🤖 DataDreamer 💤 ] Step 'documents' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Step 'documents' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' progress: 40 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity (select_columns)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'syntax describe text similarity (select_columns)' finished running l

In [24]:
all_results = concatenate_datasets(results)

In [25]:
all_results.save_to_disk('../data/described_similarities_ds')

Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

### Analyze layer to lingustic similarity:

In [26]:
all_results = load_from_disk('../data/described_similarities_ds')

In [27]:
all_results_df = all_results.to_pandas()

In [28]:
zscores= all_results_df['z-score'].tolist()

In [32]:
all_results_df[['text-1', 'text-2', 'layer', 'lexical', 'syntax', 'discourse', 'semantic']].head(n=50)

,text-1,text-2,layer,lexical,syntax,discourse,semantic
0,"Nothing Happened This Morning\n\nThe rhythms of life as captured through incredibly playful editing—the way your brain tries to make sense of reality as you’re awakened by an alarm clock and get out of bed, the excitement of morning coffee (and juice!), the way a <PERSON> flute piece can animate your living quarters (and during this the film moves from b&w to color), a ticking clock anchoring the banality of everyday as the edits show how much vibrancy exists in our lives otherwise. Loved when he’s transported between his home and the coffee shop.","Miniature\n\nThe electronic soundtrack, while moderately appropriate in its thalassic melodies, do too much heavy lifting in conveying the particular emotional pitch here. When the actual sounds of water arrive it's just too much, doubling down on the saccharine and heavy-handed. The text sours as a result, as its poeticism (already rote compared to, say, the work of <PERSON> or <PERSON> post-Benning text explorations in Fainting Spells), feels like it's not disruptive enough. This is to say that the convergence of music and poetry and dance is just too one-dimensional. I want to be invited into this film, to explore the feelings and ideas therein, but <PERSON> is spoonfeeding everything to the viewer. There's a patina of elegance but it only serves as a reminder that each element isn't treated as something expansive, but as a shortcut for vibes.",0,None,"{""reasons"": ""Both documents are film reviews and share a similar structure in terms of analyzing different aspects of the films. They both discuss the use of sound and music, the emotional impact, and the overall execution of the film. However, the tone and language used in each document are quite different. Document 1 uses more positive and enthusiastic language, while Document 2 is more critical and analytical. The sentence structures also differ, with Document 1 using shorter, more straightforward sentences and Document 2 using longer, more complex sentences."", ""score"": 3}",None,None
1,"Nothing Happened This Morning\n\nThe rhythms of life as captured through incredibly playful editing—the way your brain tries to make sense of reality as you’re awakened by an alarm clock and get out of bed, the excitement of morning coffee (and juice!), the way a <PERSON> flute piece can animate your living quarters (and during this the film moves from b&w to color), a ticking clock anchoring the banality of everyday as the edits show how much vibrancy exists in our lives otherwise. Loved when he’s transported between his home and the coffee shop.","Coffee Break\n\nThis is a silent short film with an unsuccessful structural gambit. We watch women on a coffee break and the camera remains fixed. After a while the reel runs out and we're left with an imageless screen, and then the image appears again, and then we zoom in. This repeats about five times throughout the course of the film, the interest we gain primarily coming from the way a newspaper will block another's face or looking at them flip through a photo album. But there's little consideration for how this framing is advancing the film in a meaningful way, and when the film ends with a close-up of the coffee with the women all leaving, it feels unearned.",0,None,"{""reasons"": ""Both documents discuss films, specifically focusing on the themes and cinematography. However, the first document discusses a film with a dynamic and vibrant narrative, while the second document discusses a film with a static and repetitive structure. The first document uses more descriptive and emotive language, while the second document uses more critical and analytical language. Therefore, while they share a common topic, their syntactic structures and tones are quite different."", ""score"": 2}",None,None
2,"Miniature\n\nThe electronic soundtrack, while moderately appropriate in its thalassic melodies, do too much heavy lifting in conveying the part

In [33]:
all_results_df.layer.value_counts()

0    40
2    40
3    40
6    40
4    16
1    12
5    12
Name: layer, dtype: int64

In [34]:
def aggregate_score(list_of_scores):
    scores = []
    for json_str in list_of_scores:
        if json_str == None:
            continue
        try:
            if "\n\n" in json_str:
                json_str = json_str.split("\n")[-1].strip()

            json_str = json_str.replace("Output:", "").strip()
                
                
            json_obj = json.loads(json_str)            
            scores.append(int(json_obj['score']))
        except:
            print(json_str)
            print('==============')
    return round(np.median(scores), 2)

# Aggregating layer score
layer_scores_df = all_results_df.groupby('layer').aggregate({
    'lexical': aggregate_score,
    'syntax': aggregate_score,
    'discourse': aggregate_score,
    'semantic': aggregate_score,
}).reset_index()

In [35]:
layer_scores_df.head(n=7)

,layer,lexical,syntax,discourse,semantic
0,0,1.0,3.0,2.0,1.5
1,1,2.0,3.0,3.0,3.0
2,2,2.0,3.0,3.0,2.0
3,3,1.0,2.5,2.0,1.0
4,4,1.5,2.5,2.5,2.0
5,5,2.0,3.0,3.0,2.0
6,6,1.0,2.0,1.5,1.0
